In [ ]:
import string

from torch_fuzzy_lm import FuzzyPatternLM, START_SYMBOL, END_SYMBOL
from fuzzy_lm_experiment import DatasetABA, FuzzyPatternLMExperiment

In [ ]:
train_vocab_size = 20

train_vocab = list(map(str, range(train_vocab_size)))

test_vocab = list(string.ascii_letters)

full_vocab = train_vocab + test_vocab
full_vocab += [START_SYMBOL, END_SYMBOL]

In [ ]:
def evaluate(model, split_key=""):
    split = dataset.test
    test_len = len(split[0])
    prompts = sorted({tuple(ex[: 2]) for ex in split})
    all_preds = set()
    for prompt in prompts:
        pred = tuple(model.predict_one(prompt))
        all_preds.add(pred)
    corr_key = "{}correct".format(split_key)
    n_corr_key = '{}n_correct'.format(split_key)
    incorr_key = "{}incorrect".format(split_key)
    n_incorrect_key = '{}n_incorrect'.format(split_key)
    accuracy_key = "{}accuracy".format(split_key)
    data = {
        corr_key: [],
        incorr_key: []}
    for p in all_preds:
        if dataset.is_error(p, test_len):
            data[incorr_key].append(p)
        else:
            data[corr_key].append(p)
    data[n_corr_key] = len(data[corr_key])
    data[n_incorrect_key] = len(data[incorr_key])
    data[accuracy_key] = data[n_corr_key] / len(all_preds)
    return data

In [ ]:
dataset = DatasetABA(train_vocab, test_vocab)

In [ ]:
model = FuzzyPatternLM(
    vocab=full_vocab,
    embed_dim=2,
    hidden_dim=100,
    embedding=None,
    warm_start=True,
    max_iter=150,
    eta=0.01,
    alpha=0)

In [ ]:
model.fit(dataset.train, eval_func=evaluate)

In [ ]:
preds = model.results.copy()

In [ ]:
preds[-1]['incorrect']